https://medium.com/@curiousily/tensorflow-for-hackers-part-ii-building-simple-neural-network-2d6779d2f91b

In [14]:
import pandas as pd
from math import floor
import tensorflow as tf
import numpy as np

In [84]:
data1 = pd.read_csv('premier_14_15.csv')
data2 = pd.read_csv('premier_15_16.csv')
data3 = pd.read_csv('premier_16_17.csv')
data4 = pd.read_csv('premier_17_18.csv')

dataCon = [data1,data2,data3,data4]
data = pd.concat(dataCon)

In [85]:
x_data = data.drop(['FTR','FTAG','FTHG'],1)
print("x_data:",x_data.shape)
x_data = pd.get_dummies(x_data)
print("x_data2:",x_data.shape)
y_data = data['FTR']
y_data = pd.get_dummies(y_data)
differences = data['FTHG']-data['FTAG']

x_data: (1520, 50)
x_data2: (1520, 530)


In [3]:
train_size = 0.9

train_cnt = floor(x_data.shape[0] * train_size)
x_train = x_data.iloc[0:train_cnt].values
y_train = y_data.iloc[0:train_cnt].values
x_test = x_data.iloc[train_cnt:].values
y_test = y_data.iloc[train_cnt:].values
print("x_train:",x_train.shape)
print("x_test:",x_test.shape)

x_train: (1314, 288)
x_test: (146, 288)


In [4]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

In [5]:
# Parameters
learning_rate = 0.01
num_epochs = 10
num_steps = 1000
batch_size = 64
display_step = 1

In [6]:
# Network Parameters
num_input = x_data.shape[1]
num_classes = y_data.shape[1]
num_hidden_1 = 100
num_hidden_2 = 100
num_hidden_3 = 100
drop_out_keep_prob = 0.7

In [7]:
def neural_network(x,weights,biases,keep_prob):
    layer_1 = tf.add(tf.matmul(x,weights['w1']),biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1,keep_prob)
    
    layer_2 = tf.add(tf.matmul(layer_1,weights['w2']),biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_2 = tf.nn.dropout(layer_2,keep_prob)
    
    layer_3 = tf.add(tf.matmul(layer_2,weights['w3']),biases['b3'])
    layer_3 = tf.nn.relu(layer_3)
    layer_3 = tf.nn.dropout(layer_3,keep_prob)
    
    layer_out = tf.add(tf.matmul(layer_3, weights['out']), biases['out'])
    return layer_out

In [8]:
# Store layers weight & bias
weights = {
    'w1': tf.Variable(tf.random_normal([num_input,num_hidden_1])),
    'w2': tf.Variable(tf.random_normal([num_hidden_1,num_hidden_2])),
    'w3': tf.Variable(tf.random_normal([num_hidden_2,num_hidden_3])),
    'out': tf.Variable(tf.random_normal([num_hidden_3, num_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'b3': tf.Variable(tf.random_normal([num_hidden_3])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

keep_prob = tf.placeholder("float")

In [9]:
# Construct model
logits = neural_network(x, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

In [10]:
# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [11]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [12]:
predictions = neural_network(x, weights, biases, keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)

In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        avg_cost = 0.0
        total_batch = int(len(x_train) / batch_size)
        x_batches = np.array_split(x_train, total_batch)
        y_batches = np.array_split(y_train, total_batch)
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                keep_prob: drop_out_keep_prob
                            })
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))

Epoch: 0001 cost= 1097704.007372284
Epoch: 0002 cost= 261.652129364
Epoch: 0003 cost= 262.984117889
Epoch: 0004 cost= 255.040109253
Epoch: 0005 cost= 255.835266113
Epoch: 0006 cost= 255.123238373
Epoch: 0007 cost= 254.384358978
Epoch: 0008 cost= 254.040427399
Epoch: 0009 cost= 253.987451935
Epoch: 0010 cost= 251.264560699
Optimization Finished!
Accuracy: 0.0
